## 6. GDA
Implementen su propia versión de un modelo GDA sobre el mismo dataset que el punto anterior y compare sus resultados. ¿Que modelo dió mejor accuracy? ¿Que similitudes o diferencias encuentran en los parámetros obtenidos? ¿Es posible sacar alguna conclusión de estos resultados?

In [125]:
import numpy as np
from numpy.linalg import inv, det
import pandas as pd

In [126]:
class GDA():
    def __init__(self):
        # Inicializamos los parámetros
        self.K = None
        self.mu = {}
        self.sigma = {}
        self.priors = {}

    def fit(self, X_train, y_train):
        ''' Método para entrenar el modelo GDA
        X_train: matriz de datos de entrenamiento
        y_train: vector de targets de entrenamiento
        '''

        # Aseguramos que los datos sean del tipo numpy array
        X, y = np.array(X_train), np.array(y_train)

        # Definimos a K como el vector de clases únicas
        self.K = np.unique(y)

        # Definimos los parámetros del modelo
        # mu: media de cada clase
        self.mu = {k: np.mean(X[y == k], axis=0) for k in self.K}

        # priors: probabilidad a priori de cada clase
        self.priors = {k: np.mean(y == k) for k in self.K}

        # sigma: matriz de covarianza de cada clase
        # Metemos una regularización, agregando un pequeño valor a la diagonal
        # para evitar problemas de singularidad, y asegurar invertibilidad
        self.sigma = {k: np.cov(X[y == k], rowvar=False) + 1e-6 * np.eye(X.shape[1]) for k in self.K}
    
    def predict(self, X_test):
        ''' Método para predecir la clase de los datos de test
        X_test: matriz de datos de testeo
        '''

        # Aseguramos que los datos sean del tipo numpy array
        X = np.array(X_test)
        Predict = []
        for x in X:
            # Calculamos la probabilidad de cada clase
            prob_conjunta = np.array([1/np.sqrt(det(self.sigma[k])) * np.exp(-0.5 * np.dot(np.dot((x-self.mu[k]), inv(self.sigma[k])), (x-self.mu[k]).T)) for k in self.K])
            probs = {k: prob_conjunta[k] * self.priors[k] for k in self.K}
            Predict.append(max(probs, key=probs.get))
            
        return np.array(Predict)

In [127]:
# Copiamos la clase LDA para comparar con GDA
class LDA():
    def __init__(self):
        # Inicializo los parámetros
        self.phi = []
        self.mu = []
        self.sigma = []
        self.K = []


    def fit(self, X_train, y_train):
        ''' Método para entrenar el modelo LDA
        X_train: matriz de datos de entrenamiento
        y_train: vector de targets de entrenamiento
        '''

        # Aseguramos que los datos sean del tipo numpy array
        X, y = np.array(X_train), np.array(y_train)

        # Definimos a K como el vector de clases únicas
        self.K = np.unique(y)

        # n = filas, m = columnas
        n, m = X.shape

        # Iteramos sobre las clases
        # y calculamos la probabilidad a priori phi y la media mu
        for i in self.K:
            self.phi.append(len(y[y == i]) / n)
            self.mu.append(np.mean(X[y == i], axis=0))
        # Aseguramos que phi y mu sean numpy arrays
        self.phi = np.array(self.phi)
        self.mu = np.array(self.mu)

        # Calculamos la matriz de covarianza sigma
        self.sigma = np.zeros((m, m))
        for i in range(n):
            self.sigma += (X[i] - self.mu[y[i]]).reshape(-1, 1).dot((X[i] - self.mu[y[i]]).reshape(1, -1)) / n
        # Metemos una regularización, agregando un pequeño valor a la diagonal
        # para evitar problemas de singularidad, y asegurar invertibilidad
        self.sigma += 1e-6 * np.eye(self.sigma.shape[0])
    
    def predict(self, X_test):
        ''' Método para predecir la clase de los datos de test
        X_test: matriz de datos de testeo
        '''

        # Aseguramos que los datos sean del tipo numpy array
        X = np.array(X_test)

        # Inicializamos el vector de predicciones
        Predict = []
        for j in range(X.shape[0]):
            Probas = []
            for i in range(len(self.K)):
                # Calculamos la probabilidad a posteriori
                Posterior = np.exp(-0.5 * (X[j] - self.mu[i]).dot(inv(self.sigma)).dot((X[j] - self.mu[i]).T))
                # La multiplicamos por la probabilidad a priori
                Probas.append(self.phi[i] * Posterior)
            Probas = np.array(Probas)
            # Realizamos la predicción tomando la clase con mayor probabilidad
            # y la agregamos al vector de predicciones
            Predict.append(self.K[np.argmax(Probas)])
            
        return np.array(Predict)

In [128]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Cargamos el dataset
df = pd.read_csv("Datos/dataset.csv")
df = df.dropna()

# Separar features y labels
X = df.drop(columns=["label"])
y = df["label"].map({"'Male'": 0, "'Female'": 1})  # Convertir a valores numéricos

# Hago un resample para balancear las clases
X_resampled, y_resampled = RandomOverSampler().fit_resample(X, y)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

GDA_Prueba = GDA()
GDA_Prueba.fit(X_train,y_train)
# Accuracy
print('GDA:'+ str(np.mean(GDA_Prueba.predict(X_test) == y_test)))

LDA_Prueba = LDA()
LDA_Prueba.fit(X_train, y_train)
# Accuracy
print('LDA:'+ str(np.mean(LDA_Prueba.predict(X_test) == y_test)))


GDA:0.76
LDA:0.89


En todas las pruebas, las predicciones que realiza GDA son menos acertadas que las que devuele LDA. Esto puede deberse a que este último asume que las clases comparten la misma matriz de covarianza, lo que resulta en una frontera de decisión lineal. En este dataset, esa suposición parece ajustarse mejor a los datos, permitiendo que LDA generalice mejor y obtenga mayor accuracy. Por otro lado, GDA estima una matriz de covarianza diferente para cada clase, lo que puede llevar a sobreajuste si el número de muestras no es suficientemente grande o si las clases no son realmente tan diferentes en su dispersión. Además, los parámetros obtenidos por LDA suelen ser más estables y menos sensibles al ruido, mientras que GDA puede capturar mejor relaciones no lineales si existen, pero en este caso no aporta ventajas claras. En conclusión, la simplicidad de LDA lo hace más robusto para este problema en particular.